# hw3: Обучение без учителя

*Спасибо ещё одному великому курсу mlcourse.ai и авторам: Ольга Дайховская (@aiho в Slack ODS), Юрий Кашницкий (@yorko в Slack ODS).*

### О задании

В этом задании мы разберемся с тем, как работают методы снижения размерности и кластеризации данных. Заодно еще раз попрактикуемся в
задаче классификации.

Мы будем работать с набором данных [Samsung Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Данные поступают с акселерометров и гироскопов мобильных телефонов Samsung Galaxy S3 (подробнее про признаки – по ссылке на UCI выше), также известен вид активности человека с телефоном в кармане – ходил ли он, стоял, лежал, сидел или шел вверх/вниз по лестнице.

Вначале мы представим, что вид активности нам неизвестнен, и попробуем кластеризовать людей чисто на основе имеющихся признаков. Затем решим задачу определения вида физической активности именно как задачу классификации.

**Заполните код в клетках (где написано "Ваш код здесь") и ответьте на вопросы, выделив ответ полужирным** (``` **выделить двойными звёздочками** ```).

### Оценивание и штрафы
Вам необходимо ответить на 10 вопросов и выполнить 2 задания. Каждое из заданий и вопросов имеет определенную «стоимость» (указана в скобках). Максимально допустимая оценка за работу — 10 баллов. Неэффективная и/или неоригинальная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Заполненный ноутбук ```hw3-unsupervised.ipynb``` необходимо загрузить на свой Github. Затем нужно оставить комментарий в Google-таблице с оценками в столбце "hw3" в строке со своей фамилией о том, что вы выполнили работу и оставить ссылку на ноутбук.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

%matplotlib inline
from matplotlib import pyplot as plt

plt.style.use(['seaborn-v0_8-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

RANDOM_STATE = 17

In [ ]:
try:
    X_train = np.loadtxt("../../data/samsung_HAR/samsung_train.txt")
    y_train = np.loadtxt("../../data/samsung_HAR/samsung_train_labels.txt").astype(int)
    X_test = np.loadtxt("../../data/samsung_HAR/samsung_test.txt")
    y_test = np.loadtxt("../../data/samsung_HAR/samsung_test_labels.txt").astype(int)
except FileNotFoundError:
    try:
        X_train = np.loadtxt("datasets/samsung_HAR/samsung_train.txt")
        y_train = np.loadtxt("datasets/samsung_HAR/samsung_train_labels.txt").astype(int)
        X_test = np.loadtxt("datasets/samsung_HAR/samsung_test.txt")
        y_test = np.loadtxt("datasets/samsung_HAR/samsung_test_labels.txt").astype(int)
    except FileNotFoundError:
        print("Файлы данных не найдены. Пожалуйста, скачайте данные Samsung HAR и поместите их в папку datasets/samsung_HAR/")
        print("Ссылка: https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones")

In [ ]:
# Проверим размерности
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризации нам не нужен вектор ответов, поэтому будем работать с объединением обучающей и тестовой выборок. Объедините *X_train* с *X_test*, а *y_train* – с *y_test*.

In [ ]:
X = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])

Определим число уникальных значений меток целевого класса.

In [ ]:
np.unique(y)

In [ ]:
n_classes = np.unique(y).size

[Эти метки соответствуют:](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names)
- 1 - ходьбе
- 2 - подъему вверх по лестнице
- 3 - спуску по лестнице
- 4 - сидению
- 5 - стоянию
- 6 - лежанию

*уж простите, если звучание этих существительных кажется корявым :)*

Отмасштабируйте выборку с помощью `StandardScaler` с параметрами по умолчанию.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Понижаем размерность с помощью PCA, оставляя столько компонент, сколько нужно для того, чтобы объяснить как минимум 90% дисперсии исходных (отмасштабированных) данных. Используйте отмасштабированную выборку и зафиксируйте random_state (константа RANDOM_STATE).

In [ ]:
pca = PCA(n_components=0.9, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

**Вопрос 1:** (1 балл)

Какое минимальное число главных компонент нужно выделить, чтобы объяснить 90% дисперсии исходных (отмасштабированных) данных?

**Варианты:**
- 56
- **65**
- 66
- 193

In [ ]:
n_components_90 = pca.n_components_
print(f"Количество компонент для 90% дисперсии: {n_components_90}")

**Вопрос 2:** (0.5 баллов)

Сколько процентов дисперсии приходится на первую главную компоненту? Округлите до целых процентов.

**Варианты:**
- 45
- **51**
- 56
- 61

In [ ]:
first_component_variance = pca.explained_variance_ratio_[0] * 100
print(f"Дисперсия первой главной компоненты: {first_component_variance:.2f}%")
print(f"Округлено до целых: {int(round(first_component_variance))}%")

Визуализируйте данные в проекции на первые две главные компоненты.

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, s=20, cmap='viridis')
plt.xlabel('Первая главная компонента')
plt.ylabel('Вторая главная компонента')
plt.title('Визуализация данных в проекции на первые две главные компоненты')
plt.colorbar(label='Активность')
plt.show()

**Вопрос 3:** (0.5 баллов)

Если все получилось правильно, Вы увидите сколько-то кластеров, почти идеально отделенных друг от друга. Какие виды активности входят в эти кластеры?<br>

**Ответ:**
- 1 кластер: все 6 активностей
- **2 кластера: (ходьба, подъем вверх по лестнице, спуск по лестнице) и (сидение, стояние, лежание)**
- 3 кластера: (ходьба), (подъем вверх по лестнице, спуск по лестнице) и (сидение, стояние, лежание)
- 6 кластеров

------------------------------

**Задание 1.** (1 балл)

Сделайте кластеризацию данных методом `KMeans` (собственная имплементация и готовая реализация), обучив модель на данных со сниженной за счет PCA размерностью. В данном случае мы подскажем, что нужно искать именно 6 кластеров, но в общем случае мы не будем знать, сколько кластеров надо искать.

Параметры:

- **n_clusters** = n_classes (число уникальных меток целевого класса)
- **n_init** = 100
- **random_state** = RANDOM_STATE (для воспроизводимости результата)

Остальные параметры со значениями по умолчанию.

In [ ]:
kmeans = KMeans(n_clusters=n_classes, n_init=100, random_state=RANDOM_STATE)
cluster_labels = kmeans.fit_predict(X_pca)

Визуализируйте данные в проекции на первые две главные компоненты. Раскрасьте точки в соответствии с полученными метками кластеров.

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, s=20, cmap='viridis')
plt.xlabel('Первая главная компонента')
plt.ylabel('Вторая главная компонента')
plt.title('Кластеризация KMeans (6 кластеров)')
plt.colorbar(label='Кластер')
plt.show()

Посмотрите на соответствие между метками кластеров и исходными метками классов и на то, какие виды активностей алгоритм `KMeans` путает.

In [ ]:
tab = pd.crosstab(y, cluster_labels, margins=True)
tab.index = ['ходьба', 'подъем вверх по лестнице',
             'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'все']
tab.columns = ['cluster' + str(i + 1) for i in range(6)] + ['все']
tab

max_fractions = []
activity_names = ['ходьба', 'подъем вверх по лестнице', 'спуск по лестнице', 
                  'сидение', 'стояние', 'лежание']
for i in range(6):
    class_counts = tab.iloc[i, :6].values
    max_fraction = class_counts.max() / tab.iloc[i, 6]
    max_fractions.append(max_fraction)
    print(f"{activity_names[i]}: максимальная доля = {max_fraction:.4f}")

best_activity_idx = np.argmax(max_fractions)
print(f"\nЛучше всего отделилась активность: {activity_names[best_activity_idx]} (доля = {max_fractions[best_activity_idx]:.4f})")

Видно, что kMeans не очень хорошо отличает только активности друг от друга. Используйте метод локтя, чтобы выбрать оптимальное количество кластеров. Параметры алгоритма и данные используем те же, что раньше, меняем только `n_clusters`.

In [ ]:
inertia = []
for k in tqdm(range(1, n_classes + 1)):
    kmeans_k = KMeans(n_clusters=k, n_init=100, random_state=RANDOM_STATE)
    kmeans_k.fit(X_pca)
    inertia.append(kmeans_k.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, n_classes + 1), inertia, marker='o')
plt.xlabel('Количество кластеров (k)')
plt.ylabel('Inertia')
plt.title('Метод локтя для выбора оптимального количества кластеров')
plt.grid(True)
plt.show()

**Вопрос 5:** (1 балл)

Какое количество кластеров оптимально выбрать, согласно методу локтя?<br>

**Ответ:**
- 1
- **2**
- 3
- 4

------------------------

Попробуем еще один метод кластеризации, который описывался в статье – агломеративную кластеризацию.

In [ ]:
ag = AgglomerativeClustering(n_clusters=n_classes,
                             linkage='ward').fit(X_pca)

Посчитайте Adjusted Rand Index (`sklearn.metrics`) для получившегося разбиения на кластеры и для `KMeans` с параметрами из задания к 4 вопросу.

In [ ]:
ari_kmeans = metrics.adjusted_rand_score(y, cluster_labels)
ari_agglomerative = metrics.adjusted_rand_score(y, ag.labels_)

print(f"Adjusted Rand Index для KMeans: {ari_kmeans:.4f}")
print(f"Adjusted Rand Index для Agglomerative Clustering: {ari_agglomerative:.4f}")

**Вопрос 6:** (1 балл)

Отметьте все верные утверждения.<br>

**Варианты:**
- Согласно ARI, KMeans справился с кластеризацией хуже, чем Agglomerative Clustering
- **Для ARI не имеет значения какие именно метки присвоены кластерам, имеет значение только разбиение объектов на кластеры**
- **В случае случайного разбиения на кластеры ARI будет близок к нулю**

-------------------------------

Можно заметить, что задача не очень хорошо решается именно как задача кластеризации, если выделять несколько кластеров (> 2). Давайте теперь решим задачу классификации, вспомнив, что данные у нас размечены.  

Для классификации используйте метод опорных векторов – класс `sklearn.svm.LinearSVC`. Мы в курсе отдельно не рассматривали этот алгоритм, но он очень известен, почитать про него можно, например, в материалах Евгения Соколова –  [тут](https://github.com/esokolov/ml-course-msu/blob/master/ML16/lecture-notes/Sem11_linear.pdf).

Настройте для `LinearSVC` гиперпараметр `C` с помощью `GridSearchCV`.

- Обучите новый `StandardScaler` на обучающей выборке (со всеми исходными признаками), прмиените масштабирование к тестовой выборке
- В `GridSearchCV` укажите  cv=3.

In [ ]:
scaler_train = StandardScaler()
X_train_scaled = scaler_train.fit_transform(X_train)
X_test_scaled = scaler_train.transform(X_test)

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
best_svc = GridSearchCV(svc, svc_params, cv=3, scoring='accuracy', n_jobs=-1)
best_svc.fit(X_train_scaled, y_train)

In [ ]:
print(f"Лучший параметр C: {best_svc.best_params_['C']}")
print(f"Лучший score на кросс-валидации: {best_svc.best_score_:.4f}")
print(f"\nРезультаты для всех значений C:")
for mean_score, params in zip(best_svc.cv_results_['mean_test_score'], best_svc.cv_results_['params']):
    print(f"  C={params['C']}: {mean_score:.4f}")

**Вопрос 7** (0.5 баллов)

Какое значение гиперпараметра `C` было выбрано лучшим по итогам кросс-валидации?<br>

**Ответ:**
- 0.001
- 0.01
- **0.1**
- 1
- 10

In [ ]:
y_predicted = best_svc.predict(X_test_scaled)

In [ ]:
tab = pd.crosstab(y_test, y_predicted, margins=True)
tab.index = ['ходьба', 'подъем вверх по лестнице', 'спуск по лестнице',
             'сидение', 'стояние', 'лежание', 'все']
tab.columns = tab.index
tab

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score

report = classification_report(y_test, y_predicted, 
                            target_names=['ходьба', 'подъем вверх по лестнице', 
                                        'спуск по лестнице', 'сидение', 'стояние', 'лежание'],
                            output_dict=True)
print(classification_report(y_test, y_predicted, 
                            target_names=['ходьба', 'подъем вверх по лестнице', 
                                        'спуск по лестнице', 'сидение', 'стояние', 'лежание']))

activity_names = ['ходьба', 'подъем вверх по лестнице', 'спуск по лестнице', 
                  'сидение', 'стояние', 'лежание']
precisions = [report[activity]['precision'] for activity in activity_names]
recalls = [report[activity]['recall'] for activity in activity_names]

worst_precision_idx = np.argmin(precisions)
worst_recall_idx = np.argmin(recalls)

print(f"\nХудшая точность (precision): {activity_names[worst_precision_idx]} = {precisions[worst_precision_idx]:.4f}")
print(f"Худшая полнота (recall): {activity_names[worst_recall_idx]} = {recalls[worst_recall_idx]:.4f}")


**Вопрос 8:** (0.5 балл)

Какой вид активности SVM определяет хуже всего в терминах точности? Полноты? <br>

**Ответ:**
- по точности – подъем вверх по лестнице, по полноте – лежание
- **по точности – лежание, по полноте – сидение**
- по точности – ходьба, по полноте – ходьба
- по точности – сидение, по полноте – стояние

In [ ]:
pca_train = PCA(n_components=0.9, random_state=RANDOM_STATE)
X_train_pca = pca_train.fit_transform(X_train_scaled)
X_test_pca = pca_train.transform(X_test_scaled)

svc_pca = LinearSVC(random_state=RANDOM_STATE)
best_svc_pca = GridSearchCV(svc_pca, svc_params, cv=3, scoring='accuracy', n_jobs=-1)
best_svc_pca.fit(X_train_pca, y_train)

print(f"Лучший параметр C (с PCA): {best_svc_pca.best_params_['C']}")
print(f"Лучший score на кросс-валидации (с PCA): {best_svc_pca.best_score_:.4f}")
print(f"Лучший score на кросс-валидации (без PCA): {best_svc.best_score_:.4f}")
diff_percent = (best_svc.best_score_ - best_svc_pca.best_score_) * 100
print(f"Разница: {diff_percent:.1f}%")
print(f"Округлено до целых: {int(round(diff_percent))}%")


**Вопрос 9:** (1 балл)

Какова разность между лучшим качеством (долей верных ответов) на кросс-валидации в случае всех 561 исходных признаков и во втором случае, когда применялся метод главных компонент? Округлите до целых процентов.<br>

**Варианты:**
- Качество одинаковое
- **2%**
- 4%
- 10%
- 20%


**Вопрос 10:** (1 балл)

Выберите все верные утверждения:

**Варианты:**
- Метод главных компонент в данном случае позволил уменьшить время обучения модели, при этом качество (доля верных ответов на кросс-валидации) очень пострадало, более чем на 10%
- **PCA можно использовать для визуализации данных, однако для этой задачи есть и лучше подходящие методы, например, tSNE. Зато PCA имеет меньшую вычислительную сложность**
- **PCA строит линейные комбинации исходных признаков, и в некоторых задачах они могут плохо интерпретироваться человеком**

In [ ]:
print("="*80)
print("ЗАДАНИЕ 2: DBSCAN и tSNE")
print("="*80)

print("\n1. Применение tSNE для снижения размерности...")
tsne = TSNE(n_components=2, random_state=RANDOM_STATE, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

print(f"Размерность после tSNE: {X_tsne.shape}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, s=20, cmap='viridis')
plt.xlabel('tSNE компонента 1')
plt.ylabel('tSNE компонента 2')
plt.title('Визуализация данных с помощью tSNE (истинные метки)')
plt.colorbar(label='Активность')

print("\n2. Кластеризация с помощью DBSCAN на исходных данных...")
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"Количество кластеров: {n_clusters_dbscan}")
print(f"Количество шумовых точек: {n_noise}")

plt.subplot(1, 2, 2)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=dbscan_labels, s=20, cmap='viridis')
plt.xlabel('tSNE компонента 1')
plt.ylabel('tSNE компонента 2')
plt.title(f'DBSCAN кластеризация (кластеров: {n_clusters_dbscan}, шум: {n_noise})')
plt.colorbar(label='Кластер')
plt.tight_layout()
plt.show()

if n_clusters_dbscan > 0:
    ari_dbscan = metrics.adjusted_rand_score(y, dbscan_labels)
    print(f"\nAdjusted Rand Index для DBSCAN: {ari_dbscan:.4f}")
    
    print("\nСравнение методов кластеризации:")
    print(f"  KMeans ARI: {ari_kmeans:.4f}")
    print(f"  Agglomerative Clustering ARI: {ari_agglomerative:.4f}")
    print(f"  DBSCAN ARI: {ari_dbscan:.4f}")
else:
    print("\nDBSCAN не нашел кластеров при данных параметрах.")

print("\n3. Попробуем DBSCAN на данных с PCA...")
dbscan_pca = DBSCAN(eps=2.0, min_samples=10)
dbscan_pca_labels = dbscan_pca.fit_predict(X_pca)

n_clusters_dbscan_pca = len(set(dbscan_pca_labels)) - (1 if -1 in dbscan_pca_labels else 0)
n_noise_pca = list(dbscan_pca_labels).count(-1)

print(f"Количество кластеров (на PCA данных): {n_clusters_dbscan_pca}")
print(f"Количество шумовых точек: {n_noise_pca}")

if n_clusters_dbscan_pca > 0:
    ari_dbscan_pca = metrics.adjusted_rand_score(y, dbscan_pca_labels)
    print(f"Adjusted Rand Index для DBSCAN (на PCA): {ari_dbscan_pca:.4f}")
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=dbscan_pca_labels, s=20, cmap='viridis')
    plt.xlabel('Первая главная компонента')
    plt.ylabel('Вторая главная компонента')
    plt.title(f'DBSCAN на PCA данных (кластеров: {n_clusters_dbscan_pca})')
    plt.colorbar(label='Кластер')
    
    plt.subplot(1, 2, 2)
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=dbscan_pca_labels, s=20, cmap='viridis')
    plt.xlabel('tSNE компонента 1')
    plt.ylabel('tSNE компонента 2')
    plt.title('DBSCAN на PCA данных (визуализация через tSNE)')
    plt.colorbar(label='Кластер')
    plt.tight_layout()
    plt.show()


In [ ]:
print("="*80)
print("ЗАДАНИЕ 2: DBSCAN и tSNE")
print("="*80)

print("\n1. Применение tSNE для снижения размерности...")
tsne = TSNE(n_components=2, random_state=RANDOM_STATE, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

print(f"Размерность после tSNE: {X_tsne.shape}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, s=20, cmap='viridis')
plt.xlabel('tSNE компонента 1')
plt.ylabel('tSNE компонента 2')
plt.title('Визуализация данных с помощью tSNE (истинные метки)')
plt.colorbar(label='Активность')

print("\n2. Кластеризация с помощью DBSCAN на исходных данных...")
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"Количество кластеров: {n_clusters_dbscan}")
print(f"Количество шумовых точек: {n_noise}")

plt.subplot(1, 2, 2)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=dbscan_labels, s=20, cmap='viridis')
plt.xlabel('tSNE компонента 1')
plt.ylabel('tSNE компонента 2')
plt.title(f'DBSCAN кластеризация (кластеров: {n_clusters_dbscan}, шум: {n_noise})')
plt.colorbar(label='Кластер')
plt.tight_layout()
plt.show()

if n_clusters_dbscan > 0:
    ari_dbscan = metrics.adjusted_rand_score(y, dbscan_labels)
    print(f"\nAdjusted Rand Index для DBSCAN: {ari_dbscan:.4f}")
    
    print("\nСравнение методов кластеризации:")
    print(f"  KMeans ARI: {ari_kmeans:.4f}")
    print(f"  Agglomerative Clustering ARI: {ari_agglomerative:.4f}")
    print(f"  DBSCAN ARI: {ari_dbscan:.4f}")
else:
    print("\nDBSCAN не нашел кластеров при данных параметрах.")

print("\n3. Попробуем DBSCAN на данных с PCA...")
dbscan_pca = DBSCAN(eps=2.0, min_samples=10)
dbscan_pca_labels = dbscan_pca.fit_predict(X_pca)

n_clusters_dbscan_pca = len(set(dbscan_pca_labels)) - (1 if -1 in dbscan_pca_labels else 0)
n_noise_pca = list(dbscan_pca_labels).count(-1)

print(f"Количество кластеров (на PCA данных): {n_clusters_dbscan_pca}")
print(f"Количество шумовых точек: {n_noise_pca}")

if n_clusters_dbscan_pca > 0:
    ari_dbscan_pca = metrics.adjusted_rand_score(y, dbscan_pca_labels)
    print(f"Adjusted Rand Index для DBSCAN (на PCA): {ari_dbscan_pca:.4f}")
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=dbscan_pca_labels, s=20, cmap='viridis')
    plt.xlabel('Первая главная компонента')
    plt.ylabel('Вторая главная компонента')
    plt.title(f'DBSCAN на PCA данных (кластеров: {n_clusters_dbscan_pca})')
    plt.colorbar(label='Кластер')
    
    plt.subplot(1, 2, 2)
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=dbscan_pca_labels, s=20, cmap='viridis')
    plt.xlabel('tSNE компонента 1')
    plt.ylabel('tSNE компонента 2')
    plt.title('DBSCAN на PCA данных (визуализация через tSNE)')
    plt.colorbar(label='Кластер')
    plt.tight_layout()
    plt.show()


**Задание 2.** (1 балл)

Попробуйте использовать DBSCAN в качестве алгоритма кластеризации и метод понижения размерности tSNE.

**Решение:**